In [ ]:
import glacierml as gl
import pandas as pd
import os
from tqdm import tqdm
import geopy
pd.set_option('display.max_columns', None)


In [ ]:
df = gl.data_loader()
# df=df.drop('Zmed', axis=1)
df.name = 'df'
df

In [ ]:
history_filename, model_filename = gl.build_and_train_model(df)

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model(model_filename)

In [ ]:
train_features, test_features, train_labels, test_labels = gl.data_splitter(df)

In [ ]:
import numpy as np
y_pred = model.predict(test_features).flatten()
y_true=test_labels
loss = tf.keras.losses.mean_absolute_error(y_true, y_pred)
print(loss)

In [ ]:
import pandas as pd
model_history = pd.read_csv(history_filename)
model_history

In [ ]:
RGI_All = gl.RGI_loader()
# RGI_All = RGI_All.reset_index()
RGI_All

In [ ]:
RGI = RGI.drop(RGI_All.loc[RGI_All['Zmed']<0].index)
RGI = RGI.drop(RGI.loc[RGI['Lmax']<0].index)
RGI = RGI.drop(RGI.loc[RGI['Slope']<0].index)
RGI = RGI.drop(RGI.loc[RGI['Aspect']<0].index)
RGI = RGI.reset_index()
RGI = RGI.drop('index', axis=1)
RGI = RGI.drop('Zmed', axis=1)
RGI

In [ ]:
y_pred = model.predict(RGI).flatten()

In [ ]:
np.mean(y_pred)

In [ ]:
print(f'Zmed missing: {len(RGI_All[RGI_All.Zmed<0])}\n'
      f'Slope missing: {len(RGI_All[RGI_All.Slope<0])}\n'
      f'Aspect missing: {len(RGI_All[RGI_All.Aspect<0])}\n'
      f'Lmax missing: {len(RGI_All[RGI_All.Lmax<0])}\n')

In [ ]:
print(f'Length w/o Zmed: {np.mean(RGI_All[RGI_All.Zmed<0].Lmax)}\n'
      f'Area w/ Zmed: {np.mean(RGI_All[RGI_All.Zmed>0].Area)}\n\n'
      f'Area w/o Slope: {np.mean(RGI_All[RGI_All.Slope<0].Area)}\n'
      f'Area w/ Slope: {np.mean(RGI_All[RGI_All.Slope>0].Area)}\n\n'
      f'Area w/o Aspect: {np.mean(RGI_All[RGI_All.Aspect<0].Area)}\n'
      f'Area w/ Aspect: {np.mean(RGI_All[RGI_All.Aspect>0].Area)}\n\n'
      f'Area w/o Lmax: {np.mean(RGI_All[RGI_All.Lmax<0].Area)}\n'
      f'Area w/ Lmax: {np.mean(RGI_All[RGI_All.Lmax>0].Area)}\n\n')

In [ ]:
RGI_All

In [ ]:
tau = 100e3
rho = 1e3
g = 10
alpha = RGI_All.Slope
H = tau / rho / g / np.sin(alpha / np.pi * 180)
H_avg = np.nansum(H) / sum(~np.isnan (H))
H_avg

In [ ]:
?np.sin